# The following notebook is used for development of the maps for ESCWA redo

- please note that the exports/imports will all occur in the current folder

## External imports

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import logit, expit
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText


import warnings
warnings.filterwarnings('ignore')

#Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
#import views_runs

from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

#Data fetching of predictions, note the rewrite required to make the .py files work from a different folder, may require change if the organization of the main branch changed
import sys
import os
home = os.path.expanduser("~")
user = os.getlogin()

#MapperTools
import os
import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path
from views_mapper2.mapper2 import *
from views_mapper2.dictionary_writer import *
from views_mapper2.BBoxWriter import *
from views_mapper2.label_writer import *

In [ ]:
#define a location where the output is temporarily stored in
#for now I will ask it to store on local machine
#change the location according to preference


Desktop = home + '/Desktop/'
master_folder_name = Desktop + 'escwa_testing'

#new folderpath
def make_folders():
    make_a_folder(master_folder_name)
    print(f"{user}, folders created")

make_folders()

master_folder = Desktop+'escwa_testing/'

In [ ]:
#some universals for mapping
#essentially a selection of config2

#size of the map
width_global = 20
height_global = 20

#country map levels are the ones per earlier discussion that should have a background because of how they work and greater usefullness of actual map data
#below is the setting for both the background choice
#the transparency level
#the size of custom views experimental labels
#choice is also available for the background choice for pgm maps
#choices below are what I thought looked good for the two levels
country_background_global = 'OpenStreetMap'
country_mapping_transparency_global = 0.8
views_experimental_font_global = 18

#the following allows for changes to the textbox size
#change here to preferred textbox sizes if you do not like the appearance, especially if the width and height changes were made earlier
#note that ame and africa are close enough in size that their textbox size should be the same

def function_for_textbox(location):
    if str(location) in('globe'):
        textbox_font_size = 10
    elif str(location) in ('ame', 'africa'):
        textbox_font_size = 20
    else:textbox_font_size = 15
    return textbox_font_size

#note anything fatality related should have colormap rainbow, change here if you do not wish this option
#dictionary has been also chosen per previous discussions from standard dictionaries available
#change map color is also set at bwr
#however this is a step to change these defaults if you wish to
fatality_colormap = 'rainbow'

#dictionary for dichototomous prediciton outcome
#now with the new method
raw_dich_values=np.array([0.001,0.002,0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 0.95, .99])
dich_values = []
for i in raw_dich_values:
    temp1 = logit(i)
    dich_values.append(temp1)
dich_ticklabels=['<= 0.1%', '0.2%', '0.5%', '1%', '2%', '5%', '10%', '20%', '40%', '60%', '80%', '90%', '95%', '99%']
dichotomous_dictionary_cm = dict(zip(dich_ticklabels, dich_values))

prediction_dich_title_name = 'Predicted dichotomous fatalities'

#237 is Kenya
geo_coverage_loop = ['globe', 'ame', 'africa', 237]

In [ ]:
#getting the geometries
engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)
print(f'{user} gdf_ci_master fetched successfully')

## Fetching data

In [ ]:
#fetch the data
model_wanted = 'cflong_test'
dev_id = 'escwa001'
run_id = dev_id
model_attempt = 't01'

predstore_future =  f"cm_{dev_id}_{model_wanted}"
#
#please note that these are dichotomous predictions
predictions_df = pd.DataFrame.forecasts.read_store(predstore_future, run=dev_id)

In [ ]:
#quick look at the data
predictions_df.head()

In [ ]:
#i am not yet quite sure what the variables stand for
#but here is a map for month_id 500 of the variable step_pred_3
#assuming I understand correctly it is the prediction at month_id 500 for month_id 503 (500+3)

#i am assuming that this is also state based violence
#and unlike the dichotomous percentage output we have a strict 0 or 1 output so it will be 1 of two colors

#choose a column
variables_wanted_predictions_dich = {'step_pred_3'}

#choose month_id
month_id_wanted_prediction_fat = {500}

data= predictions_df.copy()

#logit transformation
for variable in variables_wanted_predictions_dich:
    data[variable] = np.where(data[variable] == 0, 0.000001, data[variable])
    data[variable] = np.where(data[variable] == 1, 0.999999, data[variable])
    data[variable] = logit(data[variable])

gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

#savefolder
predictions_dich_cm_folder = master_folder

#looping informationa
variable_loop = variables_wanted_predictions_dich
dictionary_run = dichotomous_dictionary_cm
cmap_run = fatality_colormap
month_loop = month_id_wanted_prediction_fat

for month in month_loop:
    for variable in variable_loop:
        for region_name in geo_coverage_loop:
            data_run = gdf.loc[month]
            var_run = variable
            var_name_run = 'sb'
            var_run_savefile = 'sb'
            bbox_run = bbox_from_cid_region(region_name)
            month_id = month

            #making the choice of textbox size
            textbox_font_size = function_for_textbox(region_name)

            title_run = f'{prediction_dich_title_name} for  variable {variable} on {str(vid2date(month_id))}, {var_name_run}'
            textbox = f'Name: {dev_id}_{model_wanted},\n{var_run}_at_month{str(month)}'
            savefile = f'{predictions_dich_cm_folder}{dev_id}_cm_{var_run_savefile}_{region_name}_{cmap_run}_month{month}_{variable}.png'

            if region_name in ('globe', 'ame', 'africa'):
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_layer(
                    gdf=data_run,
                    map_dictionary=dictionary_run,
                    cmap='binary',
                    edgecolor="black",
                    linewidth=0.5,
                    transparency = 1.0,
                    column=var_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = 1,
                    masking_location = region_name,
                    column=var_run,
                    edgecolor="black",
                    linewidth=1
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)

                map_run.save(savefile)
            else:
                map_run=Mapper2(
                    width=width_global,
                    height=height_global,
                    frame_on=True,
                    title=title_run,
                    bbox=bbox_run
                ).add_mask(
                    gdf = data_run,
                    map_dictionary = dictionary_run,
                    cmap = cmap_run,
                    transparency = country_mapping_transparency_global,
                    background = country_background_global,
                    masking_location = str(cid2name(region_name)),
                    column=var_run,
                    edgecolor="black",
                    linewidth=1, 
                    views_experimental_labels = views_experimental_font_global
                ).add_views_textbox(
                    text=textbox,
                    textsize=textbox_font_size)

                map_run.save(savefile)
print(f'{user},done')